In [2]:
import scipy as sp
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt
from currency_converter import CurrencyConverter

from manager import optimiser, input_visualise

import ipywidgets as widgets
from IPython.display import display

# remove unused currency
def trim(income_data, financial_data):
    n_currency = min(1 + len(financial_data['exchange rates']), len(financial_data['deposit options']), 
                     len(financial_data['names']), len(income_data[0]), len(income_data[1]))
    financial_data['exchange rates'] = financial_data['exchange rates'][:n_currency - 1]
    financial_data['deposit options'] = financial_data['deposit options'][:n_currency]
    financial_data['names'] = financial_data['names'][:n_currency]
    income_data = income_data[:, :n_currency, ...]     
    return (income_data, financial_data)

def instalment(total_price, down_payment, duration, annual_intr_rate, first_instalment_date, aggressive=True):
    p = total_price - down_payment
    n = duration * 12
    r = annual_intr_rate / 12
    date_list = [first_instalment_date + relativedelta(months=i) for i in range(n)]
    if aggressive:
        instalments = [p / n +(p - p * k / n) * r for k in range(n)]
    else:
        instalments = [(p * r * (1 + r) ** n) / ((1+r)**n - 1)] * n
    result = list(zip(date_list, instalments))
    return result

def index_days(list_of_pairs, start_day, finish_day):
    list_of_pairs = [p for p in list_of_pairs if p[0] >= start_day and p[0] <= finish_day]
    result = [0] * ((finish_day - start_day).days + 1)
    for d, val in list_of_pairs:
        i = (d - start_day).days
        result[i] = val
    return np.array(result)

def calculate(financial_data, income_data):
    max_days = (financial_data['last date'] - financial_data['first date']).days + 1

    print("beginning search for optimal solution, using LP \n")
    ops_cum, actual_transaction_days, ops, intr_earned = optimiser(income_data, financial_data)

    print("\n success! \n")
    
    print('payment can be made up to' , actual_transaction_days[-1])
    print('total interests earned:', np.round(intr_earned, 2))
    
    optimised_day = len(ops_cum[0])
    ops_cum = np.pad(ops_cum, ((0,0), (0, max_days - optimised_day)), mode='edge')
    income_cum = np.cumsum(income_data, axis=2)
    new_income = ([1] + financial_data['exchange rates']) @ (income_cum[0] + ops_cum)
    old_income = ([1] + financial_data['exchange rates']) @ income_cum[0]
    old_expenses = ([1] + financial_data['exchange rates']) @ income_cum[1]
    plt.plot(old_income, label='cumulative income')
    plt.plot(old_expenses, label='cumulative instalment')
    plt.plot(new_income, label='income minus deposit')
    plt.legend()
    plt.show()
    
    headers = [[financial_data['names'][i] + ' ' + str(int(op[0])) + 'M' for op in cur] for i, cur in enumerate(financial_data['deposit options'])]
    headers.append([cur + ' ex' for cur in financial_data['names'][1:]])
    headers = np.concatenate(headers)
    transaction_data = [(d, op) for d, op in zip(headers,ops) if np.max(op) > 0.01]
    
    transaction_plan = pd.DataFrame(data=np.transpose(np.array([d[1] for d in transaction_data])), columns=[d[0] for d in transaction_data], index=actual_transaction_days)
    transaction_plan = transaction_plan.replace(0, '')
    print("\n Optimal transaction plan: \n")
    display(transaction_plan)
    return

# Create widgets
total_price_input = widgets.BoundedFloatText(value=850000, min=0, max=1000000, step=1000, description='Total price')
down_payment_input = widgets.BoundedFloatText(value=100000, min=0, max=1000000, step=1000, description='Down payment')
duration_input = widgets.BoundedIntText(value=5, min=5, max=30, description='years', step=1)
annual_intr_rate_input = widgets.BoundedFloatText(value=0.03, min=0.001, max=0.2, step=0.001, description='interest rate')
first_instalment_date_input = widgets.DatePicker(value=datetime.date.today(), description='Starting Date:', disabled=False)
starting_cash_input = widgets.BoundedFloatText(value=20000, min=0, max=1000000, step=1, description='cash')
transaction_interval_select = widgets.Dropdown(
    options=['every 3 months', 'monthly', 'weekly', 'daily (very slow)'], value='monthly', description='transaction frequency', disabled=False)

spreadsheet_upload = widgets.FileUpload(accept='.xlsx, .xls', multiple=False, description='income data')

calculate_button = widgets.Button(description="Calculate!")

# Output display
output = widgets.Output()

# Set Pandas options to show all rows and columns
pd.set_option('display.max_rows', None)   # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent line-wrappin

# initialise currency converter
currency_converter = CurrencyConverter()

# Define button click function
def on_button_clicked(b):
    with output:
        output.clear_output()

        print("\n loading spreadsheet... \n")

        if spreadsheet_upload.value:
            content = spreadsheet_upload.value[0]['content']
            income_sheet = pd.read_excel(BytesIO(content), sheet_name='income')
            rates_sheet = pd.read_excel(BytesIO(content), sheet_name='rates')
            output.clear_output()
        else:
            print('no file uploaded')

        total_price = total_price_input.value
        down_payment = down_payment_input.value
        duration = duration_input.value
        annual_intr_rate = annual_intr_rate_input.value
        first_instalment_date = first_instalment_date_input.value
        starting_cash = starting_cash_input.value

        match transaction_interval_select.value:
            case 'every 3 months':
                transaction_interval = relativedelta(months=3)
            case 'monthly':
                transaction_interval = relativedelta(months=1)
            case 'weekly':
                transaction_interval = relativedelta(weeks=1)
            case 'daily (very slow)':
                transaction_interval = relativedelta(days=1)    

        # set overall time window
        today = datetime.date.today()
        first_date = today
        last_date = first_instalment_date + relativedelta(years=duration) + relativedelta(months=6)
        
        # decide the list of transaction days
        transaction_days = []
        current_ind = 0
        max_days = (last_date - first_date).days + 1
        while current_ind < max_days:
            transaction_days.append(current_ind)
            next_day = first_date + relativedelta(days=current_ind) + transaction_interval
            current_ind = (next_day - first_date).days
        
        
        # extract financial informations from spreadsheet
        intr = rates_sheet[['1 M', '3 M', '6 M', '9 M', '12 M']]
        tenures = [1,3,6,9,12]
        depo_options = []
        for i, val in intr.iterrows():
            opts = [(tenure, r) for tenure, r in zip(tenures, val) if not np.isnan(r)]
            depo_options.append(opts)

        names = list(rates_sheet['currency'])
        ex_rates = [int(currency_converter.convert(100, n, names[0]))/100 for n in names[1:]]
    
        print('current exchange rates (in ' + names[0] + '):')
        print(*names[1:], sep='\t')
        print(*ex_rates, sep='\t')
        
        # constructing inputs
        financial_data = {
            'names': names,
            'deposit options': depo_options,
            'buffer time': relativedelta(days=0),
            'exchange rates': ex_rates,
            'exchange penalty': 0.01,
            'first date': first_date,
            'last date': last_date,
            'transaction days': transaction_days
        }
        
        income_dates = np.array(income_sheet['maturation'].dt.date)
        income = [index_days(list(zip(income_dates, np.array(income_sheet[c]))), first_date, last_date) for c in names]
        income[0][0] += starting_cash
        
        future_payment_list = instalment(total_price, down_payment, duration, annual_intr_rate, first_instalment_date)
        expenses = [index_days(future_payment_list, first_date, last_date)]
        expenses = np.pad(expenses, ((0, len(names) - 1),(0,0)))
        income_data = np.round(np.array([income, expenses]), 2)
        
        income_data, financial_data = trim(income_data, financial_data)

        print("\n plots of raw cash flow: \n")
        input_visualise(income_data, financial_data)

        calculate(financial_data, income_data)



# Attach the click event to the button
calculate_button.on_click(on_button_clicked)

# Display the interface
display(total_price_input, down_payment_input, 
        duration_input,
        annual_intr_rate_input,
        first_instalment_date_input,
        starting_cash_input,
        transaction_interval_select,
        spreadsheet_upload,
        calculate_button, output)


Text(value='', description='Name:')

IntSlider(value=25, description='Age:')

Button(description='Greet', style=ButtonStyle())

Output()